In [17]:
import requests
import json
from configuration import api_key
import pandas as pd
from pathlib import Path
import re
from pandas import json_normalize 

park_visitors_csv_file = Path("../Resources/national_park_visitors.csv")

#Read the csv and create variable called park_visitors_data
park_visitors_data= pd.read_csv(park_visitors_csv_file)

#Make it into a dataframe
park_visitors_df = pd.DataFrame(park_visitors_data)

#Establish nps endpoint - with api key
endpoint = f"https://developer.nps.gov/api/v1/parks?limit=600&api_key={api_key}"

#turn the result into a json file
parks_data = requests.get(endpoint).json()

#Normalize data so it all fits into a dataframe
normalized_parks_data = json_normalize(parks_data, 'data')

#Drop the columns we dont need
all_parks_df = pd.DataFrame(normalized_parks_data).drop(columns=[
'id', 
    'url', 
    'latLong', 
    'directionsInfo', 
    'directionsUrl', 
    'addresses', 
    'images', 
    'contacts.phoneNumbers',
    'contacts.emailAddresses',
    
])

#sort by only those named national park or national park and preserve
national_parks_df = all_parks_df.loc[(all_parks_df['designation'] == "National Park") | (all_parks_df['designation'] == "National Park & Preserve")]

#Clean up the " NP" Part of the Park Name
park_visitors_df_clean = park_visitors_df.replace(' NP','', regex=True)

#rename park_visitors_df columns
park_visitors_df_clean.columns = ["Park Name", "Rank", "Visitors", "Percent Of Total"]

#create a copy to start renaming columns
national_parks_df_clean = national_parks_df.copy()

#rename columns
national_parks_df_clean.rename(columns={
    'fullName':'Full Name',
    'parkCode':'Park Code',
    'description':'Description',
    'latitude':'Latitude',
    'longitude':'Longitude',
    'activities':'Activities',
    'topics':'Topics',
    'states':'States',
    'entranceFees':'Entrance Fees',
    'entrancePasses':'Entrance Passes',
    'fees':'Fees',
    'operatingHours':'Operating Hours',
    'weatherInfo':'Weather Info',
    'designation':'Designation',
    'name':'Park Name'
}, inplace = True)

#merge visitor df and park df
combined_park_data_df = park_visitors_df_clean.merge(national_parks_df_clean, on="Park Name", how='left')

print(len(combined_park_data_df))
combined_park_data_df

63


,Park Name,Rank,Visitors,Percent Of Total,Full Name,Park Code,Description,Latitude,Longitude,Activities,Topics,States,Entrance Fees,Entrance Passes,Fees,Operating Hours,Weather Info,Designation
0,Great Smoky Mountains,1,"12,937,633",14.59%,Great Smoky Mountains National Park,grsm,Ridge upon ridge of forest straddles the borde...,35.60116374,-83.50818326,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[],"NC,TN",[],[],[],"[{'exceptions': [], 'description': 'Primary ro...",Elevations in the park range from approximatel...,National Park
1,Grand Canyon,2,"4,732,101",5.34%,Grand Canyon National Park,grca,"Grand Canyon National Park, in Northern Arizon...",36.0001165336,-112.121516363,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '69693007-2DF2-4EDE-BB3B-A25EBA72BDF5'...,AZ,"[{'cost': '35.00', 'description': 'Admits one ...","[{'cost': '70.00', 'description': 'Available t...",[],"[{'exceptions': [], 'description': 'Both Deser...",This weather varies with cold winters and mild...,National Park
2,Zion,3,"4,692,417",5.29%,Zion National Park,zion,Follow the paths where native people and pione...,37.29839254,-113.0265138,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '69693007-2DF2-4EDE-BB3B-A25EBA72BDF5'...,UT,"[{'cost': '6.00', 'description': 'Rangers may ...",[],[],"[{'exceptions': [], 'description': 'Zion Natio...",Zion is known for a wide range of weather cond...,National Park
3,Rocky Mountain,4,"4,300,424",4.85%,Rocky Mountain National Park,romo,Rocky Mountain National Park's 415 square mile...,40.3556924,-105.6972879,[{'id': '5F723BAD-7359-48FC-98FA-631592256E35'...,[{'id': '69693007-2DF2-4EDE-BB3B-A25EBA72BDF5'...,CO,"[{'cost': '30.00', 'description': '1-Day Vehic...","[{'cost': '70.00', 'description': 'RMNP Park-s...",[],"[{'exceptions': [], 'description': 'While cert...","Weather can change quickly in RMNP, due to Roc...",National Park
4,Acadia,5,"3,970,260",4.48%,Acadia National Park,acad,Acadia National Park protects the natural beau...,44.409286,-68.247501,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '00F3C3F9-2D67-4802-81AE-CCEA5D3BA370'...,ME,"[{'cost': '6.00', 'description': 'Vehicle rese...","[{'cost': '70.00', 'description': 'The Acadia ...",[],"[{'exceptions': [], 'description': 'Acadia Nat...","Located on Mount Desert Island in Maine, Acadi...",National Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Isle Royale,59,"25,454",0.03%,Isle Royale National Park,isro,"Explore a rugged, isolated island far from our...",48.01145819,-88.82780657,[{'id': '09DF0950-D319-4557-A57E-04CD2F63FF42'...,[{'id': '00F3C3F9-2D67-4802-81AE-CCEA5D3BA370'...,MI,"[{'cost': '7.00', 'description': '$7 per perso...","[{'cost': '60.00', 'description': 'Isle Royale...",[],[{'exceptions': [{'exceptionHours': {'wednesda...,Isle Royale National Park is a remote island w...,National Park
59,Lake Clark & PRES,60,"18,187",0.02%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Kobuk Valley,61,"16,925",0.02%,Kobuk Valley National Park,kova,"Caribou, sand dunes, the Kobuk River, Onion Po...",67.35631336,-159.2002293,[{'id': '071BA73C-1D3C-46D4-A53C-00D5602F7F0E'...,[{'id': '7F81A0CB-B91F-4896-B9A5-41BE9A54A27B'...,AK,[],[],[],"[{'exceptions': [], 'description': 'The Northw...","Snow, rain, and freezing temperatures can occu...",National Park
61,Gates of the Arctic & PRES,62,"9,457",0.01%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
